In [ ]:
import pandas as pd

data = pd.read_csv('/content/test.csv')
data.head(10)

,review,rating
0,Selamat Ultah utk Boss saya [USERNAME]. Wartaw...,1
1,Aku dulu nemu ig nya dia. Pas dia lg liat came...,0
2,Buka puasa makan mujigae porsi berat alias seg...,1
3,Pacar yg baik adalah pacar yg nasehatin kamu g...,1
4,Min [USERNAME] [USERNAME] mau nanya dong di jl...,1
5,Nah itu yg sy gak ngerti. Tp sy bayangkan mung...,0
6,Anjir. Ini anak macam apa? Eike aja ngidam aya...,0
7,diluar sana nona-nona lagi sibuk menyeleksi ca...,0
8,liat vlive kemaren2 do megang hp jae terus har...,1
9,"Kalo kamuuuuuu, kamuu itu specialll. Cintanya ...",1


In [ ]:
import string
import re

data['review'] = data['review'].str.lower()

def casef(text):
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    text = text.encode('ascii', 'replace').decode('ascii')
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    text= text.replace("http://", " ").replace("https://", " ")
    text = re.sub(r"\d+", "", text)
    text = text.translate(str.maketrans("","",string.punctuation))
    text = text.strip()
    text = re.sub('\s+',' ',text)
    return text

data['casefold'] = data['review'].apply(casef)
hasil_casefolding= data['casefold']
hasil_casefolding

0       selamat ultah utk boss saya username wartawan ...
1       aku dulu nemu ig nya dia pas dia lg liat camer...
2       buka puasa makan mujigae porsi berat alias seg...
3       pacar yg baik adalah pacar yg nasehatin kamu g...
4       min username username mau nanya dong di jl awa...
                              ...                        
1311    kok gak tertarik pdhal enak loh gajinya besar ...
1312    duh sakit sekali karena saya org yg jujur saya...
1313    yakin aja setiap datang pasti akan pergi dan b...
1314    apapun yang terjadi kita hadapi demi terwujud ...
1315    mau jalan kaki diluar kosan tapi takut kena ja...
Name: casefold, Length: 1316, dtype: object

In [ ]:
# ------ Tokenizing ---------
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

def word_tokenize_wrapper(text):
  return word_tokenize(text)

data['comment_tokens'] = data['casefold'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n')
print(data['comment_tokens'].head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Tokenizing Result : 

0    [selamat, ultah, utk, boss, saya, username, wa...
1    [aku, dulu, nemu, ig, nya, dia, pas, dia, lg, ...
2    [buka, puasa, makan, mujigae, porsi, berat, al...
3    [pacar, yg, baik, adalah, pacar, yg, nasehatin...
4    [min, username, username, mau, nanya, dong, di...
Name: comment_tokens, dtype: object


In [ ]:
#-- FIXING INFORMAL WORDS --

normalized_word = pd.read_excel("/content/dictionary.xlsx")

normalized_word_dict = {}

for index, row in normalized_word.iterrows():
    if row[0] not in normalized_word_dict:
        normalized_word_dict[row[0]] = row[1]

def normalized_term(document):
    return [normalized_word_dict[term] if term in normalized_word_dict else term for term in document]

data['comment_revision'] = data['comment_tokens'].apply(normalized_term)

data['comment_revision'].tail(10)

1306    [yakin, saja, setiap, datang, pasti, akan, per...
1307    [apapun, yang, terjadi, kita, hadapi, demi, te...
1308    [mau, jalan, kaki, diluar, kosan, tapi, takut,...
1309    [username, nkri, gundulmu, wong, pendiri, bang...
1310    [bukber, pertama, dengan, orang, tersayang, us...
1311    [kok, tidak, tertarik, pdhal, enak, loh, gajin...
1312    [aduh, sakit, sekali, karena, saya, orang, yan...
1313    [yakin, saja, setiap, datang, pasti, akan, per...
1314    [apapun, yang, terjadi, kita, hadapi, demi, te...
1315    [mau, jalan, kaki, diluar, kosan, tapi, takut,...
Name: comment_revision, dtype: object

In [ ]:
#-- STOPWORDS REMOVAL --
list_stopwords = {"adalah","akan","akhir","aku","saya","antara","antaranya","apabila","atau","bahwa","bahwasannya","berikut","berkata","berupa","dan","dalam","dapat","dari","demikian","dengan","di","dia","beliau","mas","pak","diri","dirinya","guna","hal","hingga","ia","ialah","ibarat","ibaratnya","ibu","ingin","inginkan","ini","itu","jadi","kami","kalian","kamu","kan","karena","kini","lalu","kita","maka","mereka","merupakan","misal","misalkan","misalnya","pertama","orang","pada","nya","saat","sendiri","sini","yaitu","yang","kalau","jika","untuk","secara","sedangkan","luar","alangkah","wkkk",
                  "wkwkw","wkwkwkw","wk","wkkw"}

list_stopwords = set(list_stopwords)

def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

data['comment_filtered'] = data['comment_revision'].apply(stopwords_removal)

print(data['comment_filtered'].head())

0    [selamat, ultah, boss, username, wartawan, heb...
1    [dulu, menemukan, ig, cocok, lg, lihat, camera...
2    [buka, puasa, makan, mujigae, porsi, berat, al...
3    [pacar, baik, pacar, nasehatin, tidak, pernah,...
4    [min, username, username, mau, nanya, dong, jl...
Name: comment_filtered, dtype: object


In [ ]:
pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 14.4 MB/s 


In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}
for document in data['comment_filtered']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)

def get_stemmed_term(document):
    return [term_dict[term] for term in document]

data['comment_stemmed'] = data['comment_filtered'].apply(get_stemmed_term)
print(data['comment_stemmed'].tail(10))

1306    [yakin, saja, tiap, datang, pasti, pergi, begi...
1307    [apa, jadi, hadap, demi, wujud, impi, hati, ya...
1308    [mau, jalan, kaki, luar, kosan, tapi, takut, k...
1309    [username, nkri, gundul, wong, diri, bangsa, j...
1310    [bukber, sayang, username, username, username,...
1311    [kok, tidak, tarik, pdhal, enak, loh, gaji, be...
1312    [aduh, sakit, sekali, jujur, bakal, blg, ada, ...
1313    [yakin, saja, tiap, datang, pasti, pergi, begi...
1314    [apa, jadi, hadap, demi, wujud, impi, hati, ya...
1315    [mau, jalan, kaki, luar, kosan, tapi, takut, k...
Name: comment_stemmed, dtype: object


In [ ]:
#install scikit-learn library
!pip install -U scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
max_features = 1000
newdata = data['comment_stemmed'].astype(str)

tfidf = TfidfVectorizer(max_features=max_features, ngram_range=(1,3), smooth_idf=False)
tfs = tfidf.fit_transform(newdata)
IDF_vector = tfidf.idf_

tfidf_mat = tfidf.fit_transform(newdata).toarray()

In [ ]:
X = tfidf_mat
Y = data['rating']

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

clasfc= MultinomialNB()
scores = cross_val_score(clasfc, X, Y, cv=10)
scores

array([0.78787879, 0.8030303 , 0.77272727, 0.83333333, 0.8030303 ,
       0.85606061, 0.6870229 , 0.72519084, 0.76335878, 0.75572519])

In [ ]:
print(scores.mean() * 100, '%')

77.87358315984271 %


In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

Y_pred = cross_val_predict(clasfc, X, Y, cv=10)
conf_mat = confusion_matrix(Y, Y_pred)
conf_mat

array([[290, 220],
       [ 71, 735]])

In [ ]:
TP = 735
TN = 290
FP = 220
FN = 71

In [ ]:
accuracy = (TN+TP)/(TN+TP+FN+FP)
print(accuracy * 100, '%')

77.88753799392097 %


In [ ]:
precision = TP/(TP+FP)
print(precision * 100, '%')

76.96335078534031 %


In [ ]:
recall = TP/(TP+FN)
print(recall * 100, '%')

91.1910669975186 %


In [ ]:
f1meas = (2*precision*recall)/(precision+recall)
print(f1meas * 100, '%')

83.47529812606473 %
